In [1]:
all_parts = spark.read.csv('gs://dataproc-staging-xxxxxxxxxxxxxxxxxxxxxx/notebooks/jupyter/CrimeDetection.csv/part-*.csv', header = True, inferSchema = True)

In [2]:
df_spark = all_parts
df_spark.show()

24/01/23 10:31:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+------+---------------+-------------------+-------------------+-----------+--------+-------+--------------------+------------+-------------+-----------+-----------------+-----------------+------------+--------------+---------------+-------------+-------------------+-------------------+--------------+---------------+--------+-------+--------------------+----------+-----------+-----+
|          X|         Y|Index_|event_unique_id|     occurrencedate|       reporteddate|premisetype|ucr_code|ucr_ext|             offence|reportedyear|reportedmonth|reportedday|reporteddayofyear|reporteddayofweek|reportedhour|occurrenceyear|occurrencemonth|occurrenceday|occurrencedayofyear|occurrencedayofweek|occurrencehour|            MCI|Division|Hood_ID|       Neighbourhood|       Lat|       Long|  FID|
+-----------+----------+------+---------------+-------------------+-------------------+-----------+--------+-------+--------------------+------------+-------------+-----------+------

In [3]:
# Convert PySpark DataFrame to Pandas DataFrame
df_pandas = df_spark.toPandas()

# Display the Pandas DataFrame
df_pandas.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,FID
0,-79.484825,43.650158,40705,GO-20151233655,2015-07-20 04:00:00,2015-07-20 04:00:00,Other,1430,100,Assault,...,201,Monday,9,Assault,D11,89,Runnymede-Bloor West Village (89),43.650158,-79.484825,40238
1,-79.549538,43.727654,40304,GO-20143048630,2014-10-05 04:00:00,2014-10-05 04:00:00,House,1420,110,Assault Bodily Harm,...,278,Sunday,18,Assault,D23,5,Elms-Old Rexdale (5),43.727654,-79.549538,40239
2,-79.386787,43.670227,40507,GO-2015651989,2015-04-20 04:00:00,2015-04-20 04:00:00,Other,1430,100,Assault,...,110,Monday,8,Assault,D53,98,Rosedale-Moore Park (98),43.670227,-79.386787,40240
3,-79.549538,43.727654,40305,GO-20143048630,2014-10-05 04:00:00,2014-10-05 04:00:00,House,1420,100,Assault With Weapon,...,278,Sunday,18,Assault,D23,5,Elms-Old Rexdale (5),43.727654,-79.549538,40241
4,-79.318680,43.699272,40508,GO-2015656222,2015-04-20 04:00:00,2015-04-20 04:00:00,Apartment,1430,100,Assault,...,110,Monday,22,Assault,D54,60,Woodbine-Lumsden (60),43.699272,-79.318680,40242


In [4]:
import psycopg2

# Replace these values with your actual connection details
host = 'xxxxxxxxxxxxxxx.region.redshift-serverless.amazonaws.com'
port = '5439' # Default port for AWS Redshift
database = 'yourdatabasename'
user = 'useronyourdatabase'

In [5]:
# Establish a connection to Redshift Serverless
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password = 'xxxxxxxxxxx' # use other ways for secure it
    )
    print("Connected to Redshift Serverless successfully!")
except Exception as e:
    print(f"Error: Unable to connect to Redshift Serverless - {e}")

Connected to Redshift Serverless successfully!


In [6]:
df_pandas = df_spark.toPandas()
df_pandas.dtypes

X                             float64
Y                             float64
Index_                          int32
event_unique_id                object
occurrencedate         datetime64[ns]
reporteddate           datetime64[ns]
premisetype                    object
ucr_code                        int32
ucr_ext                         int32
offence                        object
reportedyear                    int32
reportedmonth                  object
reportedday                     int32
reporteddayofyear               int32
reporteddayofweek              object
reportedhour                    int32
occurrenceyear                  int32
occurrencemonth                object
occurrenceday                   int32
occurrencedayofyear             int32
occurrencedayofweek            object
occurrencehour                  int32
MCI                            object
Division                       object
Hood_ID                         int32
Neighbourhood                  object
Lat         

In [11]:
# Create a cursor
cursor = connection.cursor()

schema_name = 'yourschema'
# Create a new table
table_name = 'crimedetection'
create_table_query = f'''
    CREATE TABLE {schema_name}.{table_name} (
        X DOUBLE PRECISION,
        Y DOUBLE PRECISION,
        Index_ INT,
        event_unique_id VARCHAR,
        occurrencedate TIMESTAMP,
        reporteddate TIMESTAMP,
        premisetype VARCHAR,
        ucr_code INT,
        ucr_ext INT,
        offence VARCHAR,
        reportedyear INT,
        reportedmonth VARCHAR,
        reportedday INT,
        reporteddayofyear INT,
        reporteddayofweek VARCHAR,
        reportedhour INT,
        occurrenceyear INT,
        occurrencemonth VARCHAR,
        occurrenceday INT,
        occurrencedayofyear INT,
        occurrencedayofweek VARCHAR,
        occurrencehour INT,
        MCI VARCHAR,
        Division VARCHAR,
        Hood_ID INT,
        Neighbourhood VARCHAR,
        Lat DOUBLE PRECISION,
        Long DOUBLE PRECISION,
        FID INT
    );
'''

try:
    cursor.execute(create_table_query)
    connection.commit() 
    print(f"Table '{table_name}' created successfully!")
except Exception as e:
    connection.rollback()
    print(f"Error: Unable to create the table - {e}")

Error: Unable to create the table - Relation "crimedetection" already exists



In [47]:
f"INSERT INTO {schema_name}.{table_name} ({', '.join(df_pandas.columns)}) VALUES ({', '.join(['%s']*len(df_pandas.columns))});"

'INSERT INTO bigschema.crimedetection (X, Y, Index_, event_unique_id, occurrencedate, reporteddate, premisetype, ucr_code, ucr_ext, offence, reportedyear, reportedmonth, reportedday, reporteddayofyear, reporteddayofweek, reportedhour, occurrenceyear, occurrencemonth, occurrenceday, occurrencedayofyear, occurrencedayofweek, occurrencehour, MCI, Division, Hood_ID, Neighbourhood, Lat, Long, FID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'

In [19]:
cursor = connection.cursor()

schema_name = 'yourschema'
table_name = 'crimedetection'
insert_data_query = f"INSERT INTO {schema_name}.{table_name} ({', '.join(df_pandas.columns)}) VALUES ({', '.join(['%s']*len(df_pandas.columns))});"

# Identify columns with integer types (int32 or int64)
int_columns = df_pandas.select_dtypes(include=['int32', 'int64']).columns

# Convert identified columns to float
df_pandas[int_columns] = df_pandas[int_columns].astype(float)

# Specify the number of rows to insert
rows_to_insert = 100

try:
    # Iterate through the first 100 rows and insert the data
    for index, row in df_pandas.head(rows_to_insert).iterrows():
        data_to_insert = tuple(row)
        cursor.execute(insert_data_query, data_to_insert)

    connection.commit()
    print(f"{rows_to_insert} rows inserted successfully!")
except Exception as insert_error:
    connection.rollback()
    print(f"Error: Unable to insert data - {insert_error}")


100 rows inserted successfully!
